In [ ]:
import os
import warnings
warnings.filterwarnings("ignore", category=UserWarning)  
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import VarianceThreshold

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, Pool

from tqdm import tqdm


In [ ]:
def has_torch_cuda():
    try:
        import torch
        return torch.cuda.is_available()
    except Exception:
        return False

def get_nvidia_gpu_free_memory_mb():
    try:
        import pynvml
        pynvml.nvmlInit()
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)
        info = pynvml.nvmlDeviceGetMemoryInfo(handle)
        free_mb = info.free // (1024 * 1024)
        total_mb = info.total // (1024 * 1024)
        pynvml.nvmlShutdown()
        return free_mb, total_mb
    except Exception:
        return None, None


In [ ]:
GPU_AVAILABLE = has_torch_cuda()
FREE_GPU_MB, TOTAL_GPU_MB = get_nvidia_gpu_free_memory_mb()
if FREE_GPU_MB is not None:
    print(f"[GPU] Free memory detected: {FREE_GPU_MB} MB / {TOTAL_GPU_MB} MB")
else:
    print(f"[GPU] Could not detect GPU memory via pynvml. Torch reports CUDA available: {GPU_AVAILABLE}")

CATBOOST_GPU_MIN_MB = 6000  
CATBOOST_USE_GPU = False
if GPU_AVAILABLE and FREE_GPU_MB is not None and FREE_GPU_MB >= CATBOOST_GPU_MIN_MB:
    CATBOOST_USE_GPU = True
elif GPU_AVAILABLE and FREE_GPU_MB is None:
    CATBOOST_USE_GPU = True

USE_GPU_FOR_LIGHTGBM = GPU_AVAILABLE
USE_GPU_FOR_XGBOOST = GPU_AVAILABLE

print(f"CATBOOST_USE_GPU={CATBOOST_USE_GPU}, LIGHTGBM_GPU={USE_GPU_FOR_LIGHTGBM}, XGBOOST_GPU={USE_GPU_FOR_XGBOOST}")


c:\Project\Thermophysical Property Melting Point\.venv\Lib\site-packages\torch\cuda\__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


[GPU] Free memory detected: 7776 MB / 8188 MB
CATBOOST_USE_GPU=True, LIGHTGBM_GPU=True, XGBOOST_GPU=True


In [ ]:
CV_SPLITS = 5
RANDOM_STATE = 42

# Keep features manageable
TOP_K_FEATURES = 2000

LGB_PARAMS = {
    'n_estimators': 400,
    'learning_rate': 0.03,
    'max_depth': 10,
    'n_jobs': -1,
    'verbose': -1,
    'device_type': 'gpu' if USE_GPU_FOR_LIGHTGBM else 'cpu',
}

XGB_PARAMS = {
    'n_estimators': 400,
    'learning_rate': 0.03,
    'max_depth': 8,
    'tree_method': 'hist',
    'verbosity': 0,
    'random_state': RANDOM_STATE,
    'n_jobs': -1,
    'device': 'cuda' if USE_GPU_FOR_XGBOOST else 'cpu'
}

CAT_PARAMS = {
    'iterations': 500,
    'learning_rate': 0.03,
    'depth': 8,
    'verbose': 100,
    'random_state': RANDOM_STATE,
}

if not CATBOOST_USE_GPU:
    CAT_PARAMS['iterations'] = 250


In [6]:
train_df = pd.read_csv('../data/processed/train_processed.csv')
test_df = pd.read_csv('../data/processed/test_processed.csv')

print("Loaded train/test")


Loaded train/test


In [ ]:
features = [c for c in test_df.columns if c not in ['id', 'SMILES']]
X = train_df[features].copy()
y = train_df['Tm'].copy()

X = X.copy()
X.replace([np.inf, -np.inf], np.nan, inplace=True)
numeric_cols = X.select_dtypes(include=[np.number]).columns
X[numeric_cols] = X[numeric_cols].fillna(X[numeric_cols].mean())

X.columns = [c.strip().replace(' ', '_') for c in X.columns]

print(f"Initial numeric feature count: {X.shape[1]}")


Initial numeric feature count: 322


In [ ]:
vt = VarianceThreshold(threshold=1e-8)
try:
    X = pd.DataFrame(vt.fit_transform(X), columns=[c for i,c in enumerate(X.columns) if vt.get_support()[i]])
    print(f"After variance thresholding: {X.shape[1]} features remain")
except Exception:
    X = X.select_dtypes(include=[np.number])
    print(f"After forcing numeric selection: {X.shape[1]} features remain")

def select_top_k_by_corr(X_df, y_series, k=TOP_K_FEATURES):
    numeric = X_df.select_dtypes(include=[np.number]).columns
    cors = {}
    for c in numeric:
        col = X_df[c].values
        if np.nanstd(col) == 0:
            continue
        corr = np.corrcoef(col, y_series.values)[0,1]
        if np.isfinite(corr):
            cors[c] = abs(corr)
    if not cors:
        return X_df
    sel = sorted(cors.items(), key=lambda x: x[1], reverse=True)[:k]
    cols = [c for c, _ in sel]
    return X_df[cols]

if X.shape[1] > TOP_K_FEATURES:
    X = select_top_k_by_corr(X, y, k=TOP_K_FEATURES)
    print(f"Selected top {X.shape[1]} features by correlation")

print(f"Using {X.shape[1]} features for modeling")


After variance thresholding: 307 features remain
Using 307 features for modeling


In [ ]:
# Reset indices for safe iloc slicing
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

USE_LOG_TARGET = False
if (y > 0).all():
    USE_LOG_TARGET = True
    print("TARGET: all positive -> log-transform ENABLED")
else:
    print("TARGET: contains non-positive values -> log-transform DISABLED (to avoid -inf/nan)")


TARGET: all positive -> log-transform ENABLED


In [ ]:
kf = KFold(n_splits=CV_SPLITS, shuffle=True, random_state=RANDOM_STATE)
results = {}

def evaluate_model_cv(name, model, scale=False, use_log=USE_LOG_TARGET, lgb_params=None, xgb_params=None, cat_params=None):
    mae_list = []
    steps = []
    if scale:
        steps.append(('scaler', StandardScaler()))
    steps.append(('model', model))
    pipe = Pipeline(steps)
    
    it = tqdm(kf.split(X, y), total=kf.get_n_splits(), desc=f'Training {name}', leave=True)
    for fold_idx, (tr_idx, val_idx) in enumerate(it):
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

        if use_log:
            y_tr_fit = np.log(y_tr)
            pipe.fit(X_tr, y_tr_fit)
            y_pred_log = pipe.predict(X_val)
            y_pred = np.exp(y_pred_log)
        else:
            if name == 'LightGBM' and isinstance(model, lgb.LGBMRegressor):
                model.set_params(**(lgb_params or {}))
                model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], early_stopping_rounds=50, verbose=False)
                y_pred = model.predict(X_val, num_iteration=model.best_iteration_)
            elif name == 'XGBoost' and isinstance(model, xgb.XGBRegressor):
                model.set_params(**(xgb_params or {}))
                model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], early_stopping_rounds=50, verbose=False)
                y_pred = model.predict(X_val)
            elif name == 'CatBoost' and isinstance(model, CatBoostRegressor):
                model.set_params(**(cat_params or {}))
                model.fit(X_tr, y_tr, eval_set=(X_val, y_val), use_best_model=True, verbose=cat_params.get('verbose', False))
                y_pred = model.predict(X_val)
            else:
                pipe.fit(X_tr, y_tr)
                y_pred = pipe.predict(X_val)

        if np.any(np.isnan(y_pred)):
            y_pred = np.nan_to_num(y_pred, nan=np.nanmedian(y_pred))
        mae = mean_absolute_error(y_val.values, y_pred)
        mae_list.append(mae)
        it.set_postfix_str(f"fold {fold_idx+1} mae {mae:.5f}")
        print(f"{name} Fold {fold_idx+1} MAE: {mae:.5f}")

    mean_mae = np.mean(mae_list)
    std_mae = np.std(mae_list)
    results[name] = {'mean_mae': mean_mae, 'std_mae': std_mae}
    print(f"{name} - Average MAE: {mean_mae:.5f} +/- {std_mae:.5f}\n")


In [ ]:
models = [
    ('Ridge', Ridge(alpha=1.0, random_state=RANDOM_STATE), True, {}),
    ('ElasticNet', ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=RANDOM_STATE), True, {}),
    ('RandomForest', RandomForestRegressor(n_estimators=200, max_depth=12, random_state=RANDOM_STATE, n_jobs=-1), False, {}),
    ('HistGB', HistGradientBoostingRegressor(max_iter=300, max_depth=10, random_state=RANDOM_STATE), False, {}),

    ('LightGBM', lgb.LGBMRegressor(**LGB_PARAMS), False, LGB_PARAMS),

    ('XGBoost', xgb.XGBRegressor(**XGB_PARAMS), False, XGB_PARAMS),

    ('CatBoost', CatBoostRegressor(task_type='GPU' if CATBOOST_USE_GPU else 'CPU', devices='0' if CATBOOST_USE_GPU else None,
                                  iterations=CAT_PARAMS['iterations'],
                                  learning_rate=CAT_PARAMS['learning_rate'],
                                  depth=CAT_PARAMS['depth'],
                                  random_seed=RANDOM_STATE,
                                  verbose=CAT_PARAMS.get('verbose', 100),
                                  allow_writing_files=False),
     False, CAT_PARAMS),

    ('KNN', KNeighborsRegressor(n_neighbors=5, n_jobs=-1), True, {}),
]


In [ ]:
for name, model, scale, extra_params in models:
    print(f"=== Evaluating: {name} ===")
    try:
        if name == 'LightGBM':
            evaluate_model_cv(name, model, scale=scale, use_log=USE_LOG_TARGET, lgb_params=extra_params)
        elif name == 'XGBoost':
            evaluate_model_cv(name, model, scale=scale, use_log=USE_LOG_TARGET, xgb_params=extra_params)
        elif name == 'CatBoost':
            evaluate_model_cv(name, model, scale=scale, use_log=USE_LOG_TARGET, cat_params=extra_params)
        else:
            evaluate_model_cv(name, model, scale=scale, use_log=USE_LOG_TARGET)
    except Exception as e:
        print(f"[Error] {name} failed: {e}")
        continue

print("Final results summary:")
for k, v in results.items():
    print(f"{k}: mean MAE = {v['mean_mae']:.5f}, std = {v['std_mae']:.5f}")


=== Evaluating: Ridge ===


Training Ridge:   0%|          | 0/5 [00:00<?, ?it/s, fold 3 mae 0.11809]

Ridge Fold 1 MAE: 0.12490
Ridge Fold 2 MAE: 0.11782
Ridge Fold 3 MAE: 0.11809


Training Ridge: 100%|██████████| 5/5 [00:00<00:00, 46.58it/s, fold 5 mae 0.13041]


Ridge Fold 4 MAE: 0.12626
Ridge Fold 5 MAE: 0.13041
Ridge - Average MAE: 0.12349 +/- 0.00487

=== Evaluating: ElasticNet ===


...

499:	learn: 0.0190845	total: 10.9s	remaining: 0us
CatBoost Fold 5 MAE: 0.10294
CatBoost - Average MAE: 0.10779 +/- 0.00268

=== Evaluating: KNN ===


Training KNN:  40%|████      | 2/5 [00:00<00:00, 16.95it/s, fold 2 mae 0.14316]

KNN Fold 1 MAE: 0.13920
KNN Fold 2 MAE: 0.14316


Training KNN: 100%|██████████| 5/5 [00:00<00:00, 23.98it/s, fold 5 mae 0.14622]

KNN Fold 3 MAE: 0.13404
KNN Fold 4 MAE: 0.13499
KNN Fold 5 MAE: 0.14622
KNN - Average MAE: 0.13952 +/- 0.00467

Final results summary:
Ridge: mean MAE = 0.12349, std = 0.00487
ElasticNet: mean MAE = 0.24502, std = 0.00348
RandomForest: mean MAE = 0.11108, std = 0.00227
HistGB: mean MAE = 0.10132, std = 0.00255
LightGBM: mean MAE = 0.10085, std = 0.00269
XGBoost: mean MAE = 0.10490, std = 0.00319
CatBoost: mean MAE = 0.10779, std = 0.00268
KNN: mean MAE = 0.13952, std = 0.00467


In [ ]:
from sklearn.base import clone
from xgboost import callback
def ensemble_top3(models, X, y, kf):
    oof_preds = np.zeros(len(y))
    mae_list = []

    it = tqdm(kf.split(X, y), total=kf.get_n_splits(), desc="Ensemble Top 3", leave=True)
    for fold_idx, (tr_idx, val_idx) in enumerate(it):
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

        fold_preds = []
        for name, model, params in models:
            m = clone(model)
            
            if name == 'LightGBM':
                m.set_params(**params)
                m.fit(
                    X_tr, y_tr,
                    eval_set=[(X_val, y_val)],
                    callbacks=[
                        lgb.early_stopping(stopping_rounds=50),
                        lgb.log_evaluation(0)
                    ]
                )
                fold_preds.append(m.predict(X_val, num_iteration=m.best_iteration_))
            
            elif name == 'XGBoost':
                m.set_params(**params)
                m.fit(X_tr, y_tr)
                fold_preds.append(m.predict(X_val))
            
            else:  # HistGradientBoosting
                m.fit(X_tr, y_tr)
                fold_preds.append(m.predict(X_val))

        y_pred = np.mean(fold_preds, axis=0)
        oof_preds[val_idx] = y_pred

        mae = mean_absolute_error(y_val, y_pred)
        mae_list.append(mae)
        print(f"Ensemble Fold {fold_idx+1} MAE: {mae:.5f}")

    mean_mae = np.mean(mae_list)
    std_mae = np.std(mae_list)
    print(f"Ensemble - Average MAE: {mean_mae:.5f} +/- {std_mae:.5f}")
    return oof_preds, mean_mae, std_mae


top3_models = [
    ('LightGBM', lgb.LGBMRegressor(**LGB_PARAMS), LGB_PARAMS),
    ('HistGB', HistGradientBoostingRegressor(max_iter=300, max_depth=10, random_state=RANDOM_STATE), {}),
    ('XGBoost', xgb.XGBRegressor(**XGB_PARAMS), XGB_PARAMS),
]

ensemble_oof, ens_mean, ens_std = ensemble_top3(top3_models, X, y, kf)

results['EnsembleTop3'] = {'mean_mae': ens_mean, 'std_mae': ens_std}

print("\n=== Final Results with Ensemble ===")
for k, v in results.items():
    print(f"{k}: mean MAE = {v['mean_mae']:.5f}, std = {v['std_mae']:.5f}")



Ensemble Top 3:   0%|          | 0/5 [00:00<?, ?it/s]

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[390]	valid_0's l2: 0.0222541
Ensemble Fold 1 MAE: 0.10209
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[396]	valid_0's l2: 0.0189421
Ensemble Fold 2 MAE: 0.09902
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[399]	valid_0's l2: 0.0217741
Ensemble Fold 3 MAE: 0.09920
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[400]	valid_0's l2: 0.0189577
Ensemble Fold 4 MAE: 0.10172
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[399]	valid_0's l2: 0.0187803
Ensemble Fold 5 MAE: 0.09687
Ensemble - Average MAE: 0.09978 +/- 0.00192

=== Final Results with Ensemble ===
Ridge: mean MAE = 0.12349, std = 0.00487
ElasticNet: mean MAE = 0.24502, std = 

In [ ]:
# Feature Importance
print("\n=== Feature Importances (LightGBM) ===")
lgb_model = lgb.LGBMRegressor(**LGB_PARAMS)
lgb_model.fit(X, y)
lgb_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': lgb_model.feature_importances_
}).sort_values(by='importance', ascending=False)
print(lgb_importance.head(15))

print("\n=== Feature Importances (XGBoost) ===")
xgb_model = xgb.XGBRegressor(**XGB_PARAMS)
xgb_model.fit(X, y)
xgb_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': xgb_model.feature_importances_
}).sort_values(by='importance', ascending=False)
print(xgb_importance.head(15))



=== Feature Importances (LightGBM) ===
              feature  importance
163         PEOE_VSA7         261
121  FpDensityMorgan2         260
117  MinPartialCharge         247
132          BalabanJ         244
205       VSA_EState8         240
129       BCUT2D_MRHI         234
122  FpDensityMorgan3         231
127     BCUT2D_LOGPHI         221
186              TPSA         216
128    BCUT2D_LOGPLOW         215
133           BertzCT         209
130      BCUT2D_MRLOW         207
120  FpDensityMorgan1         202
204       VSA_EState7         176
150            Kappa3         176

=== Feature Importances (XGBoost) ===
                    feature  importance
133                 BertzCT    0.120085
209               NHOHCount    0.106629
230               RingCount    0.055161
113              ExactMolWt    0.042661
221              NumHDonors    0.040694
186                    TPSA    0.040424
112          HeavyAtomMolWt    0.028706
210                 NOCount    0.021037
55               